In [ ]:
#Install tqdm to have progress bar
!pip install tqdm

#install necessary pkg to make connection with elasticsearch domain
!pip install "elasticsearch<7.14.0"
!pip install requests
!pip install requests-aws4auth
!pip install "sagemaker>=2.0.0<3.0.0"

If you run this notebook in SageMaker Studio, you need to make sure `ipywidgets` is installed and restart the kernel, so please uncomment the code in the next cell, and run it.


In [ ]:
# %%capture
# import IPython
# import sys

# !{sys.executable} -m pip install ipywidgets
# IPython.Application.instance().kernel.do_shutdown(True)  # has to restart kernel so changes are used

In [ ]:
import boto3
import re
import time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

s3_resource = boto3.resource("s3")
s3 = boto3.client('s3')

print(f'SageMaker SDK Version: {sagemaker.__version__}')

In [ ]:
cfn = boto3.client('cloudformation')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "nlu-search"

outputs = get_cfn_outputs(cloudformation_stack_name)

bucket = outputs['s3BucketTraining']
es_host = outputs['esHostName']

outputs

In [ ]:
## Data Preparation

import os 
import shutil
import json
import tqdm
import urllib.request
from tqdm import notebook
from multiprocessing import cpu_count
from tqdm.contrib.concurrent import process_map

# images_path = 'data/feidegger/fashion'
# filename = 'metadata.json'

# my_bucket = s3_resource.Bucket(bucket)

# os.makedirs(images_path, exist_ok=True)

# def download_metadata(url):
#     if not os.path.exists(filename):
#         urllib.request.urlretrieve(url, filename)
        
# #download metadata.json to local notebook
# download_metadata('https://raw.githubusercontent.com/zalandoresearch/feidegger/master/data/FEIDEGGER_release_1.1.json')

# def generate_image_list(filename):
#     metadata = open(filename,'r')
#     data = json.load(metadata)
#     url_lst = []
#     for i in range(len(data)):
#         url_lst.append(data[i]['url'])
#     return url_lst


# def download_image(url):
#     urllib.request.urlretrieve(url, images_path + '/' + url.split("/")[-1])
                    
# #generate image list            
# url_lst = generate_image_list(filename)     

# workers = 2 * cpu_count()

# #downloading images to local disk
# process_map(download_image, url_lst, max_workers=workers)


In [ ]:
# Uploading dataset to S3

# files_to_upload = []
# dirName = 'data'
# for path, subdirs, files in os.walk('./' + dirName):
#     path = path.replace("\\","/")
#     directory_name = path.replace('./',"")
#     for file in files:
#         files_to_upload.append({
#             "filename": os.path.join(path, file),
#             "key": directory_name+'/'+file
#         })
        

# def upload_to_s3(file):
#         my_bucket.upload_file(file['filename'], file['key'])
        
# #uploading images to s3
# process_map(upload_to_s3, files_to_upload, max_workers=workers)

## Lauange Translation

This dataset has product descriptions in German, So we will use Amazon Translate for English translation for each German sentence.

In [ ]:
with open("movie_data.json") as json_file:
    results = json.load(json_file)

In [ ]:
#Define translator function
# def translate_txt(data):
#     results = {}
#     results['filename'] = f's3://{bucket}/data/feidegger/fashion/' + data['url'].split("/")[-1]
#     results['descriptions'] = []
#     translate = boto3.client(service_name='translate', use_ssl=True)
#     for i in data['descriptions']:
#         result = translate.translate_text(Text=str(i), 
#             SourceLanguageCode="de", TargetLanguageCode="en")
#         results['descriptions'].append(result['TranslatedText'])
#     return results

In [ ]:
# we are using realtime traslation which will take around ~35 min. 
# workers = 1 * cpu_count()

# #downloading images to local disk
# results = process_map(translate_txt, data, max_workers=workers)

In [ ]:
# Saving the translated text in json format in case you need later time
# with open('zalando-translated-data.json', 'w', encoding='utf-8') as f:
#     json.dump(results, f, ensure_ascii=False, indent=4)

In [ ]:
!pip install install transformers[torch]

In [ ]:
#Save the model to disk which we will host at sagemaker
from transformers import AutoTokenizer, AutoModel
saved_model_dir = 'transformer'
os.makedirs(saved_model_dir, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/distilbert-base-nli-stsb-mean-tokens")
model = AutoModel.from_pretrained("sentence-transformers/distilbert-base-nli-stsb-mean-tokens") 

tokenizer.save_pretrained(saved_model_dir)
model.save_pretrained(saved_model_dir)

In [ ]:
#Defining default bucket for SageMaker pretrained model hosting
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
#zip the model in tar.gz format
!cd transformer && tar czvf ../model.tar.gz *

In [ ]:
#Upload the model to S3

inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='sentence-transformers-model')
inputs

First we need to create a PyTorchModel object. The deploy() method on the model object creates an endpoint which serves prediction requests in real-time. If the instance_type is set to a SageMaker instance type (e.g. ml.m5.large) then the model will be deployed on SageMaker. If the instance_type parameter is set to *__local__* then it will be deployed locally as a Docker container and ready for testing locally.

First we need to create a [`Predictor`](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html) class to accept TEXT as input and output JSON. The default behaviour is to accept a numpy array.

In [ ]:
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role

class StringPredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')
           

In [ ]:
pytorch_model = PyTorchModel(model_data = inputs, 
                             role=role, 
                             entry_point ='inference.py',
                             source_dir = './code',
                             py_version = 'py3', 
                             framework_version = '1.7.1',
                             predictor_cls=StringPredictor)

predictor = pytorch_model.deploy(instance_type='ml.g4dn.xlarge', 
                                 initial_instance_count=1, 
                                 endpoint_name = f'nlu-search-model-{int(time.time())}')


HuggingFace Transformers uses BERT pretrained model so it will generate 768 dimension for the given text. we will quickly validate the same in next cell.

In [ ]:
# Doing a quick test to make sure model is generating the embeddings
import json
payload = 'a halloween movie'
features = predictor.predict(payload)
embedding = json.loads(features)

embedding

## Build a KNN Index in Elasticsearch

KNN for Amazon Elasticsearch Service lets you search for points in a vector space and find the "nearest neighbors" for those points by cosine similarity. Use cases include recommendations (for example, an "other songs you might like" feature in a music application), image recognition, and fraud detection.

KNN cosine similarity requires Elasticsearch 7.7 or later (Default is Euclidean distance). Full documentation for the Elasticsearch feature, including descriptions of settings and statistics, is available in the Open Distro for Elasticsearch documentation. For background information about the k-nearest neighbors algorithm

In this step we'll get all the translated product descriptions of *__zalandoresearch__* dataset and import those embadings into Elastichseach 7.7 domain.

In [ ]:
# setting up the Elasticsearch connection
from elasticsearch import Elasticsearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
region = 'us-east-1' # e.g. us-east-1
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

es = Elasticsearch(
    hosts = [{'host': es_host, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

In [ ]:
#KNN index maping
knn_index = {
    "settings": {
        "index.knn": True,
        "index.knn.space_type": "cosinesimil",
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },
    "mappings": {
        "properties": {
           "movie_nlu_vector": { 
                "type": "knn_vector",
                "dimension": 768
            } 
        }
    }
}

In [ ]:
#Creating the Elasticsearch index
es.indices.create(index="idx_movie",body=knn_index,ignore=400) # change the index name 

In [ ]:
# If you need to load zalando-translated-data.json un-comment and execute the following code
# with open('zalando-translated-data.json') as json_file:
#     results = json.load(json_file)

In [ ]:
results[0]['description']

In [ ]:
# For each product, we are concatenating all the 
# product descriptions into a single sentence,
# so that we will have one embedding for each product

# def concat_desc(results):
#     obj = {
#         'filename': results['filename'],
#     }
#     obj['descriptions'] = ' '.join(results['descriptions'])
#     return obj

# concat_results = map(concat_desc, results)
# concat_results = list(concat_results)
# concat_results[0]

In [ ]:
# defining a function to import the feature vectors corrosponds to each S3 URI into Elasticsearch KNN index
# This process will take around ~2 min.

def es_import(concat_result):
    vector = json.loads(predictor.predict(concat_result['description']))
    es.index(index='idx_movie',
             body={"movie_nlu_vector": vector,
                   "title": concat_result['title'],
                   "description": concat_result['description']}
            )
        
workers = 4 * cpu_count()
    
process_map(es_import, results, max_workers=workers)

## Evaluate Index Search Results

In this step we will use SageMaker SDK as well as Boto3 SDK to query the Elasticsearch to retrive the nearest neighbours and retrive the relevent product images from Amazon S3 to display in the notebook.

In [ ]:
#define display_image function
# from PIL import Image
# import io
# def display_image(bucket, key):
#     s3_object = my_bucket.Object(key)
#     response = s3_object.get()
#     file_stream = response['Body']
#     img = Image.open(file_stream)
#     return display(img)

## SageMaker SDK Method

In [ ]:
#SageMaker SDK approach
import json
# payload = 'I want a dress that is yellow and flowery'
# features = predictor.predict(payload)
# embedding = json.loads(features)

In [ ]:
#ES index search
import json
# k = 3
# idx_name = 'idx_zalando'
# res = es.search(request_timeout=30, index=idx_name,
#                 body={'size': k, 
#                       'query': {'knn': {'zalando_nlu_vector': {'vector': embedding, 'k': k}}}})


In [ ]:
#Display the image

# for x in res['hits']['hits']:
#     key = x['_source']['image']
#     key = key.replace(f's3://{bucket}/','')
#     img = display_image(bucket,key)


## Boto3 Method

In [ ]:
#calling SageMaker Endpoint
client = boto3.client('sagemaker-runtime')
ENDPOINT_NAME = predictor.endpoint
response = client.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='text/plain',
                                       Body=payload)

response_body = json.loads((response['Body'].read()))


In [ ]:
#ES index search
import json
k = 3
idx_name = 'idx_movie'
res = es.search(request_timeout=30, index=idx_name,
                body={'size': k, 
                      'query': {'knn': {'movie_nlu_vector': {'vector': response_body, 'k': k}}}})

In [ ]:
#Display the name

for i in range(k):
    key = res['hits']['hits'][i]['_source']['title']
    print(key)
    # key = key.replace(f's3://{bucket}/','')
    # img = display_image(bucket,key)

## Standard Full-Text Amazon ES Search

In [ ]:
fuzzy_search_body = {
    "_source": {
        "excludes": [ "movie_nlu_vector" ]
    },
    "query": {
        "match" : {
            "description" : {
                "query" : payload
            }
        }
    }
}

result_fuzzy = es.search(request_timeout=30, index=idx_name,
                body=fuzzy_search_body)
    
for x in result_fuzzy['hits']['hits'][:3]:
    key = x['_source']['title']
    # key = key.replace(f's3://{bucket}/','')
    # img = display_image(bucket,key)

## Deploying a full-stack NLU search application

In [ ]:
s3_resource.Object(bucket, 'backend/template.yaml').upload_file('./backend/template.yaml', ExtraArgs={'ACL':'public-read'})


sam_template_url = f'https://{bucket}.s3.amazonaws.com/backend/template.yaml'

# Generate the CloudFormation Quick Create Link

print("Click the URL below to create the backend API for NLU search:\n")
print((
    'https://console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/create/review'
    f'?templateURL={sam_template_url}'
    '&stackName=nlu-search-api'
    f'&param_BucketName={outputs["s3BucketTraining"]}'
    f'&param_DomainName={outputs["esDomainName"]}'
    f'&param_ElasticSearchURL={outputs["esHostName"]}'
    f'&param_SagemakerEndpoint={predictor.endpoint}'
))

Now that you have a working Amazon SageMaker endpoint for extracting image features and a KNN index on Elasticsearch, you are ready to build a real-world full-stack ML-powered web app. The SAM template you just created will deploy an Amazon API Gateway and AWS Lambda function. The Lambda function runs your code in response to HTTP requests that are sent to the API Gateway.

In [ ]:
# Review the content of the Lambda function code.
!pygmentize backend/lambda/app.py

## Once the CloudFormation Stack shows CREATE_COMPLETE, proceed to this cell below:

In [ ]:
import json
api_endpoint = get_cfn_outputs('nlu-search-api')['TextSimilarityApi']

with open('./frontend/src/config/config.json', 'w') as outfile:
    json.dump({'apiEndpoint': api_endpoint}, outfile)

## Step 2: Deploy frontend services

In [ ]:
# add NPM to the path so we can assemble the web frontend from our notebook code

from os import environ

npm_path = ':/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin'

if npm_path not in environ['PATH']:
    ADD_NPM_PATH = environ['PATH']
    ADD_NPM_PATH = ADD_NPM_PATH + npm_path
else:
    ADD_NPM_PATH = environ['PATH']
    
%set_env PATH=$ADD_NPM_PATH

In [ ]:
%cd ./frontend/

!npm install

In [ ]:
!npm run-script build

In [ ]:
hosting_bucket = f"s3://{outputs['s3BucketHostingBucketName']}"

!aws s3 sync ./build/ $hosting_bucket --acl public-read

## Step 3: Browse your frontend service

In [ ]:
print('Click the URL below:\n')
print(outputs['S3BucketSecureURL'] + '/index.html')

You should see the following page:

![Website](./frontendExample.png)

In the search bar, try typing *__"I want a summery dress that's flowery and yellow"__*. Notice how the results are more relevant in the KNN Search method in contrast with the conventional full-text match search query. The KNN Search method uses features generated from the search text and finds similar product descriptions, which allows it to find similar products, even if the description doesn't have the exact same phrase.

## Extensions

We have used pretrained BERT model from sentence-transformers. You can fine tune BERT model based on your own usecase with your own data.

## Cleanup

Make sure that you stop the notebook instance, delete the Amazon SageMaker endpoint and delete the Elasticsearch domain to prevent any additional charges.

In [ ]:
# Delete the endpoint
predictor.delete_endpoint()

# Empty S3 Contents
training_bucket_resource = s3_resource.Bucket(bucket)
training_bucket_resource.objects.all().delete()

hosting_bucket_resource = s3_resource.Bucket(outputs['s3BucketHostingBucketName'])
hosting_bucket_resource.objects.all().delete()